# Нормализация данных

In [1]:
import pandas as pd

df = pd.read_csv(r'../data/pharmacy_dataset.csv')


In [2]:




# Функция для создания последовательных ID
def create_id_column(data, column_name='id'):
    return data.assign(**{column_name: range(1, len(data)+1)})

# Создание нормализованных таблиц с последовательными ID
dosage_forms = (df[['Лек. форма']]
                .drop_duplicates()
                .pipe(create_id_column, 'dosage_form_id')
                .rename(columns={'Лек. форма': 'form_name'}))

countries = (df[['Страна производителя']]
             .drop_duplicates()
             .pipe(create_id_column, 'country_id')
             .rename(columns={'Страна производителя': 'country_name'}))

countries.country_name = countries.country_name.apply(lambda x: x.upper())

active_substances = (df[['МНН / Действ. вещества']]
                    .drop_duplicates()
                    .pipe(create_id_column, 'mnn_id')
                    .rename(columns={'МНН / Действ. вещества': 'mnn_name'}))

pharmaco_groups = (df[['Фармако-терапевтическая группа']]
                   .drop_duplicates()
                   .pipe(create_id_column, 'group_id')
                   .rename(columns={'Фармако-терапевтическая группа': 'group_name'}))

# Создание таблицы products с последовательными ID
products = (df[['Торговое наименование', 'МНН / Действ. вещества', 
               'Лек. форма', 'Страна производителя', 
               'Фармако-терапевтическая группа', 'ЖНВЛП', 
               'ПККН', 'Характер', 'flag']]
            .drop_duplicates()
            .pipe(create_id_column, 'product_id')
            .rename(columns={
                'Торговое наименование': 'trade_name',
                'ЖНВЛП': 'is_essential',
                'ПККН': 'is_pkkn',
                'Характер': 'characteristic'
            }))

# Добавление внешних ключей с merge
products = (products
            .merge(active_substances, 
                  left_on='МНН / Действ. вещества', 
                  right_on='mnn_name')
            .merge(dosage_forms,
                  left_on='Лек. форма',
                  right_on='form_name')
            .merge(countries,
                  left_on='Страна производителя',
                  right_on='country_name')
            .merge(pharmaco_groups,
                  left_on='Фармако-терапевтическая группа',
                  right_on='group_name')
            [['product_id', 'trade_name', 'mnn_id', 'dosage_form_id',
              'country_id', 'group_id', 'is_essential', 'is_pkkn',
              'characteristic', 'flag']])

products = products.drop('flag',axis=1)

# Создание таблицы фактов с последовательными ID
sales_facts = (df[['Месяц', 'Торговое наименование', 'Кол-во']]
               .merge(products[['product_id', 'trade_name']],
                     left_on='Торговое наименование',
                     right_on='trade_name')
               .rename(columns={
                   'Месяц': 'month',
                   'Кол-во': 'quantity'
               })[[ 'month', 'product_id', 'quantity']])


In [3]:
dosage_forms

,form_name,dosage_form_id
0,раствор,1
17,таблетки,2
29,мазь для наружного применения,3
80,капсулы пролонгированного действия,4
84,крем для наружного применения,5
...,...,...
8834,"капли глазные, [без консерванта]",58
9244,спрей назальный дозированный,59
9598,спрей для местного и наружного применения,60
9600,спрей для наружного применения [спиртовой],61


# SQL - Scripts 
## 1. Создание БД 

In [ ]:
# with open('../sql-scripts/pharma_db_schema.sql', 'w') as f:
#     # Записываем SQL схему из предыдущего блока
#     f.write("""
# -- Создание таблицы active_substances (Активные вещества)
# CREATE TABLE active_substances (
#     mnn_id SERIAL PRIMARY KEY,
#     mnn_name VARCHAR(255) NOT NULL
# );

# -- Создание таблицы dosage_forms (Лекарственные формы)
# CREATE TABLE dosage_forms (
#     dosage_form_id SERIAL PRIMARY KEY,
#     form_name VARCHAR(100) NOT NULL
# );

# -- Создание таблицы countries (Страны производителей)
# CREATE TABLE countries (
#     country_id SERIAL PRIMARY KEY,
#     country_name VARCHAR(100) NOT NULL
# );

# -- Создание таблицы pharmaco_groups (Фармако-терапевтические группы)
# CREATE TABLE pharmaco_groups (
#     group_id SERIAL PRIMARY KEY,
#     group_name VARCHAR(255) NOT NULL
# );

# -- Создание таблицы products (Препараты)
# CREATE TABLE products (
#     product_id SERIAL PRIMARY KEY,
#     trade_name VARCHAR(255) NOT NULL,
#     mnn_id INT NOT NULL REFERENCES active_substances(mnn_id),
#     dosage_form_id INT NOT NULL REFERENCES dosage_forms(dosage_form_id),
#     country_id INT NOT NULL REFERENCES countries(country_id),
#     group_id INT NOT NULL REFERENCES pharmaco_groups(group_id),
#     is_essential BOOLEAN,  -- ЖНВЛП
#     is_pkkn BOOLEAN,       -- ПККН
#     characteristic VARCHAR(100)  -- Характер
# );

# -- Создание таблицы sales_facts (Факты продаж/поставок)
# CREATE TABLE sales_facts (
#     month DATE NOT NULL,
#     product_id INT NOT NULL REFERENCES products(product_id),
#     quantity INT NOT NULL,
#     PRIMARY KEY (month, product_id)
# );

# -- Создание индексов для ускорения запросов
# CREATE INDEX idx_sales_facts_product ON sales_facts(product_id);
# CREATE INDEX idx_sales_facts_month ON sales_facts(month);
# CREATE INDEX idx_products_mnn ON products(mnn_id);
# CREATE INDEX idx_products_dosage_form ON products(dosage_form_id);
# CREATE INDEX idx_products_country ON products(country_id);
# CREATE INDEX idx_products_pharmaco_group ON products(pharmaco_group_id);
# """)

## 2. Создание INSERT скрипта

In [ ]:
# with open('../sql-scripts/pharma_db_schema_inserts.sql', 'w') as f:   
#     def generate_inserts(table_name, df):
#         inserts = []
#         cols = ', '.join(df.columns)
#         print(cols)
#         for _, row in df.iterrows():
#             values = []
#             for val in row:
#                 if pd.isna(val):
#                     values.append('NULL')
#                 elif isinstance(val, str):
#                     escaped_val = val.replace("'", "''")
#                     escaped_val = escaped_val.replace(';',' +')
#                     values.append(f"'{escaped_val}'")
#                 else:
#                     values.append(str(val))
#             inserts.append(f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(values)});")
#         return inserts

# # Использование

#     f.write('\n'.join(generate_inserts('active_substances', active_substances)))
#     f.write('\n'.join(generate_inserts('dosage_forms', dosage_forms)))
#     f.write('\n'.join(generate_inserts('countries', countries)))
#     f.write('\n'.join(generate_inserts('pharmaco_groups', pharmaco_groups)))
#     f.write('\n'.join(generate_inserts('products', products)))

#     # Для sales_facts обрабатываем даты специальным образом
#     f.write('\n')
    
#     sales_facts = sales_facts.drop_duplicates(['month','product_id'])
    
#     for _, row in sales_facts.iterrows():
#         month = f"'{row['month']}'"
#         f.write(f"INSERT INTO sales_facts (month, product_id, quantity) VALUES ({month}, {row['product_id']}, {row['quantity']});\n")

mnn_name, mnn_id
form_name, dosage_form_id
country_name, country_id
group_name, group_id
product_id, trade_name, mnn_id, dosage_form_id, country_id, group_id, is_essential, is_pkkn, characteristic


In [6]:
active_substances.mnn_id.nunique()

442

# Исполнение SQL - скриптов

In [9]:
from sqlalchemy import create_engine, text

# Создаем in-memory SQLite через SQLAlchemy
engine = create_engine('sqlite:///:memory:')

with engine.connect() as conn:

    with open('../sql-scripts/pharma_db_schema.sql') as f:
        for statement in f.read().split(';'):
            if statement.strip():
                conn.execute(text(statement))
    
    
    conn.commit()

In [10]:
with engine.connect() as conn:

    with open('../sql-scripts/pharma_db_schema_inserts.sql') as f:
        for statement in f.read().split(';'):
            if statement.strip():
                conn.execute(text(statement))
    
    conn.commit()

In [17]:
with engine.connect() as conn:
    sales_facts_db = pd.read_sql('SELECT * FROM sales_facts',conn)
    products_db = pd.read_sql('SELECT * FROM products',conn)


    print(sales_facts_db.shape[0], products_db.shape[0])

13563 1431
